# Amazon Product Reviews Scraping

In [24]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser
import requests
import random

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')

def scrape(url):
    user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',]
    user_agent = random.choice(user_agent_list)
    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': user_agent,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open("urls.txt",'r', encoding = "utf-8") as urllist, open('amazon_dress_reviews.csv','w', encoding = 'utf-8') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=["title","content","date","variant","images","verified","author","rating","product","url"],quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for url in urllist.readlines():
        data = scrape(url)
        if data:
            for r in data['reviews']:
                r["product"] = data["product_title"]
                r['url'] = url
                if 'verified' in r:
                    if 'Verified Purchase' in r['verified']:
                        r['verified'] = 'Yes'
                    else:
                        r['verified'] = 'Yes'
                r['rating'] = r['rating'].split(' out of')[0]
                date_posted = r['date'].split('on ')[-1]
                if r['images']:
                    r['images'] = "\n".join(r['images'])
                r['date'] = dateparser.parse(date_posted).strftime('%d %b %Y')
                writer.writerow(r)
            # sleep(5)

# Load into a DataFrame

In [2]:
import pandas as pd
reviews = pd.read_csv('amazon_dress_reviews.csv')  
reviews1 = reviews['images'].str.split("\r\n", n=4, expand=True)
#reviews1.head(10)

In [7]:
new = reviews.merge(reviews1, how = 'left', left_index = True, right_index = True)
new = new.rename(columns={0: "image1", 1: "image2", 2: "image3", 3: "image4", 4: "image5"})
dress_reviews = new.drop(['images'], axis=1)
dress_reviews.head(10)

,title,content,date,variant,verified,author,rating,product,url,image1,image2,image3,image4,image5
0,Just stellar,The dresses look just like the photo. I've ord...,05 May 2019,Size: Large Color: As Picture8-light Blue,Yes,Warbler,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...
1,"The sheerness, the cuteness and the comfort ar...",I tried this dress out because a lot of the re...,09 Jul 2019,Size: X-Small Color: As Picture16-green,Yes,Amy,4.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,None,None,None,None
2,Great dress for a great price!,I really liked this dress! May order in other ...,13 Jun 2019,Size: 3X-Large Color: As Picture14-gray,Yes,Allison,4.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,None,None,None,None
3,Very comfortable!,Product colors are more vibrant and vivid han ...,19 Jun 2019,Size: Medium Color: As Picture55-multi-colored,Yes,SRM,4.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,None,None,None,None
4,Great dress,This is a really pretty tiered maxi dress and ...,24 Jun 2019,Size: XX-Large Color: As Picture8-light Blue,Yes,Nat,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,NaN,NaN,NaN,NaN,NaN
5,Cute breathable sun dress,I love this dress it fits perfectly on my 5’1 ...,03 Jun 2019,Size: X-Large Color: As Picture8-light Blue,Yes,carrie saylor,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,None,None,None,None
6,"In Love, the perfect flowing dress for hot wea...",I read all the reviews before purchasing. I a...,29 May 2019,Size: Large Color: As Picture10-multi-colored,Yes,Colleen Marie Taylor,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,NaN,NaN,NaN,NaN,NaN
7,Darling Dress,Your browser does not support HTML5 video. I l...,08 Aug 2019,Size: X-Large Color: As Picture10-multi-colored,Yes,Pandiiland,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,None,None
8,"Not plus size, but not a skinny-mini? Then thi...",I debated back and forth to get this dress. Th...,06 Jun 2020,Size: XX-Large Color: As Picture10-multi-colored,Yes,Elle Gore,3.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,NaN,NaN,NaN,NaN,NaN
9,You will be very happy with this purchase.,I loved the dresses. They fit perfectly and ar...,26 May 2019,Size: 3X-Large Color: As Picture19-dark Green,Yes,BSC,5.0,YESNO Women Casual Loose Bohemian Floral Print...,https://www.amazon.com/YESNO-Casual-Bohemian-D...,NaN,NaN,NaN,NaN,NaN


In [8]:
print(len(dress_reviews))

250
